<a href="https://colab.research.google.com/github/BigDavies/Colab-bankdata/blob/main/Bank_FD_prediction_with_PySpark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install Java Development kit for Spark
!apt-get install openjdk-8-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-

In [2]:
import os
#Set the JAVA_HOME env variable
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
#Current working directory
!pwd

/content


In [4]:
!echo $JAVA_HOME

/usr/lib/jvm/java-8-openjdk-amd64


In [10]:
#Install PySpark with latest version
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b075878bcbcd89d1ea3c7245310015deb40bb4d116fdb9b142b2ae4b3fa1e570
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
#mount your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [11]:
#Create object of spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-pipeline').getOrCreate()

In [12]:
#Read the csv file
df=spark.read.csv('/content/drive/My Drive/bank.csv',inferSchema=True,header=True)

In [13]:
df.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|     admin.| married| tertiary|     no|    184|     no|  

In [14]:
df.toPandas().head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [15]:
df.select('default').distinct().show()

+-------+
|default|
+-------+
|     no|
|    yes|
+-------+



In [16]:
df.select('contact').distinct().show()

+---------+
|  contact|
+---------+
|  unknown|
| cellular|
|telephone|
+---------+



In [17]:
df.select('day').distinct().show()

+---+
|day|
+---+
| 31|
| 28|
| 26|
| 27|
| 12|
| 22|
|  1|
| 13|
|  6|
| 16|
|  3|
| 20|
|  5|
| 19|
| 15|
|  9|
| 17|
|  4|
|  8|
| 23|
+---+
only showing top 20 rows



In [18]:
df.select('month').distinct().show()

+-----+
|month|
+-----+
|  jun|
|  aug|
|  may|
|  feb|
|  sep|
|  mar|
|  oct|
|  jul|
|  nov|
|  apr|
|  dec|
|  jan|
+-----+



In [19]:
df.select('pdays').distinct().show()

+-----+
|pdays|
+-----+
|  148|
|  463|
|  392|
|  243|
|   31|
|   85|
|  251|
|  137|
|   65|
|  458|
|   53|
|  481|
|  804|
|  255|
|  133|
|  296|
|  472|
|   78|
|  322|
|  321|
+-----+
only showing top 20 rows



In [20]:
df.select('previous').distinct().show()

+--------+
|previous|
+--------+
|      28|
|      26|
|      27|
|      12|
|      22|
|       1|
|      13|
|       6|
|      16|
|       3|
|      20|
|      40|
|       5|
|      19|
|      41|
|      15|
|      37|
|       9|
|      17|
|       4|
+--------+
only showing top 20 rows



In [21]:
df.select('poutcome').distinct().show()

+--------+
|poutcome|
+--------+
| success|
| unknown|
|   other|
| failure|
+--------+



In [22]:
#Drop unwanted columns
df = df.drop(*['contact', 'day', 'month','default'])
df.columns

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'loan',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'deposit']

In [23]:
df.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('deposit', 'string')]

In [24]:
cols = df.columns
cols

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'loan',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'deposit']

In [25]:
for item in df.dtypes:
  print(item)

('age', 'int')
('job', 'string')
('marital', 'string')
('education', 'string')
('balance', 'int')
('housing', 'string')
('loan', 'string')
('duration', 'int')
('campaign', 'int')
('pdays', 'int')
('previous', 'int')
('poutcome', 'string')
('deposit', 'string')


In [26]:
#create a list of the columns that are string typed
categoricalColumns = [item[0] for item in df.dtypes if item[1].startswith('string') ]
categoricalColumns

['job', 'marital', 'education', 'housing', 'loan', 'poutcome', 'deposit']

In [27]:
#create a list of the columns that are string typed
numericColumns = [item[0] for item in df.dtypes if item[1].startswith('int') ]
numericColumns

['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']

In [28]:
df.select(numericColumns).describe().toPandas()

,summary,age,balance,duration,campaign,pdays,previous
0,count,11162,11162,11162,11162,11162,11162
1,mean,41.231947679627304,1528.5385235620856,371.99381831213043,2.508421429851281,51.33040673714388,0.8325568894463358
2,stddev,11.913369192215518,3225.413325946149,347.12838571630687,2.7220771816614824,108.75828197197717,2.292007218670508
3,min,18,-6847,2,1,-1,0
4,max,95,81204,3881,63,854,58


In [29]:
df.select(numericColumns).show(5)

+---+-------+--------+--------+-----+--------+
|age|balance|duration|campaign|pdays|previous|
+---+-------+--------+--------+-----+--------+
| 59|   2343|    1042|       1|   -1|       0|
| 56|     45|    1467|       1|   -1|       0|
| 41|   1270|    1389|       1|   -1|       0|
| 55|   2476|     579|       1|   -1|       0|
| 54|    184|     673|       2|   -1|       0|
+---+-------+--------+--------+-----+--------+
only showing top 5 rows



In [30]:
type(numericColumns)

list

In [31]:
numeric_data = df.select(numericColumns).toPandas()
numeric_data.corr()

,age,balance,duration,campaign,pdays,previous
age,1.000000,0.112300,0.000189,-0.005278,0.002774,0.020169
balance,0.112300,1.000000,0.022436,-0.013894,0.017411,0.030805
duration,0.000189,0.022436,1.000000,-0.041557,-0.027392,-0.026716
campaign,-0.005278,-0.013894,-0.041557,1.000000,-0.102726,-0.049699
pdays,0.002774,0.017411,-0.027392,-0.102726,1.000000,0.507272
previous,0.020169,0.030805,-0.026716,-0.049699,0.507272,1.000000


In [32]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler

In [33]:
# Define your actual lists of categorical and numeric column names
categoricalColumns = ['job', 'marital', 'education', 'housing', 'loan', 'poutcome', 'deposit']
numericColumns = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']

# Initialize an empty list to store all preprocessing stages
stages = []

# Iterate through categorical columns for encoding
for categoricalCol in categoricalColumns:
    # StringIndexer: maps a string column of labels to an ML column of label indices
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + 'Index')

    # OneHotEncoder: maps category indices to binary vectors
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])

    # Add both StringIndexer and OneHotEncoder to the stages list
    stages += [stringIndexer, encoder]

# StringIndexer for the label column
label_stringIdx = StringIndexer(inputCol='deposit', outputCol='label')
stages += [label_stringIdx]

# Apply scaler transformation on numeric columns
numericAssembler = VectorAssembler(inputCols=numericColumns, outputCol="numeric_features")
stages += [numericAssembler]

scaler = StandardScaler(inputCol="numeric_features", outputCol="scaled_numeric_features", withStd=True, withMean=True)
stages += [scaler]

# Assemble all features (encoded categorical and scaled numeric) into a single feature vector
assemblerInputs = [c + "classVec" for c in categoricalColumns] + ["scaled_numeric_features"]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]


In [34]:
from pyspark.ml import Pipeline
"""
Pipeline is a sequence of stages that are executed in a specified order to process
and transform data. It allows you to chain together multiple data processing and
modeling steps into a single workflow, making it easier to manage and apply transformations
consistently across datasets.
"""
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
# list of names of 'label', 'features' and all exixting columns
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [35]:
df.toPandas().head(5)

,label,features,age,job,marital,education,balance,housing,loan,duration,campaign,pdays,previous,poutcome,deposit
0,1.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,admin.,married,secondary,2343,yes,no,1042,1,-1,0,unknown,yes
1,1.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",56,admin.,married,secondary,45,no,no,1467,1,-1,0,unknown,yes
2,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",41,technician,married,secondary,1270,yes,no,1389,1,-1,0,unknown,yes
3,1.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",55,services,married,secondary,2476,yes,no,579,1,-1,0,unknown,yes
4,1.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",54,admin.,married,tertiary,184,no,no,673,2,-1,0,unknown,yes


In [36]:
train_df, test_df = df.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train_df.count()))
print("Test Dataset Count: " + str(test_df.count()))

Training Dataset Count: 7855
Test Dataset Count: 3307


In [37]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train_df)

In [38]:
lr_summary=lrModel.summary

In [39]:
#Overall accuracy of the classification model
lr_summary.accuracy

1.0

In [40]:
#Precision of both classes
print(lr_summary.precisionByLabel)

[1.0, 1.0]


In [41]:
#Recall of both classes
print(lr_summary.recallByLabel)

[1.0, 1.0]


In [42]:
predictions = lrModel.transform(test_df)
predictions.show(10)

+-----+--------------------+---+----------+-------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+--------------------+--------------------+----------+
|label|            features|age|       job|marital|education|balance|housing|loan|duration|campaign|pdays|previous|poutcome|deposit|       rawPrediction|         probability|prediction|
+-----+--------------------+---+----------+-------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+--------------------+--------------------+----------+
|  0.0|(28,[0,11,13,16,1...| 31|management|married|secondary|   1064|     no|  no|      60|       1|   -1|       0| unknown|     no|[8.19678521568445...|[0.99972453795751...|       0.0|
|  0.0|(28,[0,11,13,16,1...| 40|management|married|secondary|     99|     no|  no|      15|       5|   -1|       0| unknown|     no|[8.48288856029387...|[0.99979306286755...|       0.0|
|  0.0|(28,[0,11,13,16,1...| 42|management|married|secondary|    -34| 

In [43]:
predictions.select('label','prediction').toPandas().head(10)

,label,prediction
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [44]:
#Evaluate our Logistic Regression model.
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test data result', evaluator.evaluate(predictions))

Test data result 0.9999996339023955
